In [1]:
import requests
from datetime import datetime
import psycopg2
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# This is for regression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor

from sklearn import tree
import joblib

In [17]:
conn = psycopg2.connect(database="postgres", user="postgres",
                        password="Aa@123456", host="127.0.0.1", port="5432")

In [2]:
ARIMA_URL = "http://localhost:9000/predict/ARIMA"
VARMAX_URL = "http://localhost:9000/predict/VARMAX"
SES_URL = "http://localhost:9000/predict/SES"
ACTUAL_URL = "http://localhost:9000/btc/price"

In [20]:
actual_res = requests.get(url=ACTUAL_URL)
actual_data = actual_res.json()


ARIMA_res = requests.get(url=ARIMA_URL)
ARIMA_data = ARIMA_res.json()[:-2]

VARMAX_res = requests.get(url=VARMAX_URL)
VARMAX_data = VARMAX_res.json()[:-2]

SES_res = requests.get(url=SES_URL)
SES_data = SES_res.json()[:-2]

actual = pd.DataFrame.from_dict(actual_data)
actual.columns = ['datetime', 'PRICE']
               
arima = pd.DataFrame.from_dict(ARIMA_data)
arima.columns = ['datetime', 'ARIMA']

varmax = pd.DataFrame.from_dict(VARMAX_data)
varmax.columns = ['datetime', 'VARMAX']

ses = pd.DataFrame.from_dict(SES_data)
ses.columns = ['datetime', 'SES']

arima_df = actual.merge(arima, on='datetime')
arima_varmax_df = arima_df.merge(varmax, on='datetime')
all_models_df = arima_varmax_df.merge(ses, on='datetime')

X = all_models_df.drop(["datetime", "PRICE"], axis=1)
y = all_models_df.PRICE

reg_forest = RandomForestRegressor(
    n_estimators = 100,
    criterion = 'mse')

reg_forest.fit(X, y)

ARIMA_preds = ARIMA_res.json()[-2:]
VARMAX_preds = VARMAX_res.json()[-2:]
SES_preds = SES_res.json()[-2:]

for i in range(len(ARIMA_preds)):
    arima_pred = ARIMA_preds[i]["price"]
    varmax_pred = VARMAX_preds[i]["price"]
    ses_pred = SES_preds[i]["price"]

    current_time = ARIMA_preds[i]["datetime"]
    res = reg_forest.predict([[arima_pred, varmax_pred, ses_pred]])[0]
    print("DateTime: {}, Price: {}".format(current_time, res))

DateTime: 18:50:00, Price: 34241.749866833336
DateTime: 18:50:30, Price: 34237.21787333332
